In [1]:
from model_builder import model_builder
from neural_network.classes.Callbacks import EarlyStopping
from neural_network.classes.LossFunctions import MeanEuclideanDistance
from neural_network.classes.Optimizers import *
from neural_network.classes.Validation import *
import pandas as pd
from neural_network.classes.ActivationFunctions import Sigmoid, TanH
from neural_network.classes.Validation import TunerHO

Importing the dataset

In [2]:
dataset_attribute_columns = ["a1", "a2", "a3", "a4", "a5", "a6", "a7", "a8", "a9"]
dataset_class_column = ["target_x", "target_y"]
dataset_train = pd.read_csv("neural_network/datasets/MLCup/train.csv", skiprows=7, index_col=0, names= dataset_attribute_columns + dataset_class_column)
dataset_test = pd.read_csv("neural_network/datasets/MLCup/test.csv", skiprows=7, index_col=0, names= dataset_attribute_columns + dataset_class_column)

train_set_y = dataset_train[dataset_class_column].to_numpy()
train_set_x = dataset_train[dataset_attribute_columns].to_numpy()

test_set_y = dataset_test[dataset_class_column].to_numpy()
test_set_x = dataset_test[dataset_attribute_columns].to_numpy()

Hyperparameter values for coarse-grained search

In [3]:
hp = {"num_hidden_layers": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[2, 3, 4],
    unfold=True),
    "neurons_in_layer_1": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[10, 70],
    unfold=True),
    "neurons_in_layer_2": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[10, 70],
    unfold=True),
    "neurons_in_layer_3": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[0, 10, 70],
    unfold=True),
    "neurons_in_layer_4": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[0, 10, 70],
    unfold=True),
    "loss_function": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[MeanEuclideanDistance],
    unfold=True),
    "optimizer": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[SGD],
    unfold=True),
    "learning_rate": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[0.1, 0.01, 0.001],
    unfold=True),
    "momentum": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[0.9, 0.5, 0.1],
    unfold=True),
    "regularization": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[1e-8, 1e-5, 1e-2],
    unfold=True),
    "activation_function": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[Sigmoid, TanH],
    unfold=True),
    "batch_size": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[200],
    unfold=True),
    "num_epochs": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[1000],
    unfold=True),
    "callbacks": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[[EarlyStopping(monitor="val_mean_euclidean_distance", patience=50, mode="min", min_delta=1e-2, restore_best_weight=False)]],
    unfold=True)
}

Here we perform a random search with few trials to have some results in short time. Running the whole grid search require much more time

In [4]:
# tuner = TunerHO(ConfigurationGenerator(hp, mode="grid"), model_builder, val_size=0.25, verbose=True,
#                 default_metric="val_mean_euclidean_distance", default_reverse=False, shuffle=True)
tuner = TunerHO(ConfigurationGenerator(hp, mode="random", num_trials=16), model_builder, val_size=0.25, verbose=False,
                default_metric="val_mean_euclidean_distance", default_reverse=False, shuffle=True)

results = tuner.fit(train_set_x, train_set_y)
results.sort(metric="val_mean_euclidean_distance", reverse=False)
results.dump("./coarse_search.pickle")

Hyperparameter values for fine-grained search

In [5]:
hp = {"num_hidden_layers": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[3],
    unfold=True),
    "neurons_in_layer_1": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[10, 15, 20],
    unfold=True),
    "neurons_in_layer_2": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[10, 15, 20],
    unfold=True),
    "neurons_in_layer_3": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[10, 15, 20],
    unfold=True),
    "loss_function": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[MeanEuclideanDistance],
    unfold=True),
    "optimizer": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[SGD],
    unfold=True),
    "learning_rate": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[0.05, 0.03, 0.01, 0.005],
    unfold=True),
    "momentum": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[0.6, 0.5, 0.4],
    unfold=True),
    "regularization": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[0., 1e-8, 1e-7, 1e-6],
    unfold=True),
    "activation_function": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[Sigmoid],
    unfold=True),
    "batch_size": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[200],
    unfold=True),
    "num_epochs": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[1000],
    unfold=True),
    "callbacks": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[[EarlyStopping(monitor="val_mean_euclidean_distance", patience=50, mode="min", min_delta=1e-2, restore_best_weight=False)]],
    unfold=True)
}

Again we perform a random search to finish in short time

In [7]:
# tuner = TunerCV(ConfigurationGenerator(hp, mode="grid"), model_builder, n_fold=4, verbose=True,
#                 default_metric="val_mean_euclidean_distance", default_reverse=False)
tuner = TunerCV(ConfigurationGenerator(hp, mode="random", num_trials=16), model_builder, n_fold=4, verbose=False,
                default_metric="val_mean_euclidean_distance", default_reverse=False)

val_results = tuner.fit(train_set_x, train_set_y)
best_model = tuner.best_model()
best_model.verbose = True
refit_results = best_model.fit(train_set_x, train_set_y, epochs=1000, batch_size=200)
test_result = best_model.evaluate_result(test_set_x, test_set_y)
test_result.refit_result = refit_results
test_result.validation_results = val_results

print("Training error:", test_result.validation_results.list[0].metrics["mean_euclidean_distance"])
print("Validation error:", test_result.validation_results.list[0].metrics["val_mean_euclidean_distance"])
print("Training error of refit", test_result.refit_result.metrics["mean_euclidean_distance"])
print("Test error", test_result.metrics["mean_euclidean_distance"])

test_result.dump("./test_results.pickle")

Epoch  1 mse: 794.4973    mean_euclidean_distance: 27.5897    
Epoch  2 mse: 500.6204    mean_euclidean_distance: 21.6338    
Epoch  3 mse: 402.0349    mean_euclidean_distance: 19.2146    
Epoch  4 mse: 309.6162    mean_euclidean_distance: 16.6249    
Epoch  5 mse: 224.7809    mean_euclidean_distance: 13.8366    
Epoch  6 mse: 159.1869    mean_euclidean_distance: 11.4284    
Epoch  7 mse: 117.5118    mean_euclidean_distance: 9.8099    
Epoch  8 mse: 92.5258    mean_euclidean_distance: 8.7643    
Epoch  9 mse: 78.0494    mean_euclidean_distance: 8.0966    
Epoch  10 mse: 69.9954    mean_euclidean_distance: 7.6799    
Epoch  11 mse: 65.7130    mean_euclidean_distance: 7.4275    
Epoch  12 mse: 63.4929    mean_euclidean_distance: 7.2766    
Epoch  13 mse: 62.3039    mean_euclidean_distance: 7.1836    
Epoch  14 mse: 61.5733    mean_euclidean_distance: 7.1211    
Epoch  15 mse: 61.0018    mean_euclidean_distance: 7.0728    
Epoch  16 mse: 60.4351    mean_euclidean_distance: 7.0290    
Epoc

Once the model assessment is done, we merged the train and test set and performed a final model selection with final refit to achive the best performance possible on the ML Cup

In [8]:
full_dataset_x = np.concatenate([train_set_x, test_set_x], axis=0)
full_dataset_y = np.concatenate([train_set_y, test_set_y], axis=0)

final_val = tuner.fit(full_dataset_x, full_dataset_y)
best_model = tuner.best_model()
final_refit = best_model.fit(full_dataset_x, full_dataset_y)

print("Training error:", final_val.list[0].metrics["mean_euclidean_distance"])
print("Validation error:", final_val.list[0].metrics["val_mean_euclidean_distance"])
print("Training error of refit:", final_refit.metrics["mean_euclidean_distance"])

final_val.dump("./final_val_results.pickle")
final_refit.dump("./final_refit_results.pickle")
best_model.dump_weights("./best_weights.pickle")

Training error: 1.372746997676584
Validation error: 1.43371104253514
Training error of refit: 1.3142635743249733


Prediction on the blind test set

In [9]:
dataset = pd.read_csv("neural_network/datasets/ML-CUP22-TS.csv", skiprows=7, index_col=0, names= dataset_attribute_columns)
blind_test_set = dataset[dataset_attribute_columns].to_numpy()

prediction = best_model.predict(blind_test_set)
df = pd.DataFrame(prediction)
df.to_csv("./final_prediction.csv", float_format='%.16f', index=True, header=False)